In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/14 21:49:10 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/14 21:49:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/14 21:49:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# create dataframe

In [3]:
data = (("James","Sales","NY",9000,34),
("Alicia","Sales","NY",8600,56),
("Robert","Sales","CA",8100,30),
("Lisa","Finance","CA",9000,24),
("Deja","Finance","CA",9900,40),
("Sugie","Finance","NY",8300,36),
("Ram","Finance","NY",7900,53),
("Kyle","Marketing","CA",8000,25),
("Reid","Marketing","NY",9100,50)
)
schema=("empname","dept","state","salary","age")
df = spark.createDataFrame(data=data,schema=schema)

In [4]:
df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



# groupby
- count(), min(), max(), avg(), sum()

In [9]:
from pyspark.sql.functions import col
df.groupBy('dept').count().show()

+---------+-----+
|     dept|count|
+---------+-----+
|    Sales|    3|
|  Finance|    4|
|Marketing|    2|
+---------+-----+



In [10]:
df.groupBy('dept').min().show()

+---------+-----------+--------+
|     dept|min(salary)|min(age)|
+---------+-----------+--------+
|    Sales|       8100|      30|
|  Finance|       7900|      24|
|Marketing|       8000|      25|
+---------+-----------+--------+



In [11]:
df.groupBy('dept').max().show()

+---------+-----------+--------+
|     dept|max(salary)|max(age)|
+---------+-----------+--------+
|    Sales|       9000|      56|
|  Finance|       9900|      53|
|Marketing|       9100|      50|
+---------+-----------+--------+



In [12]:
df.groupBy('dept').sum().show()

+---------+-----------+--------+
|     dept|sum(salary)|sum(age)|
+---------+-----------+--------+
|    Sales|      25700|     120|
|  Finance|      35100|     153|
|Marketing|      17100|      75|
+---------+-----------+--------+



# groupBy with mulitple columns

In [25]:
df.groupBy('dept','state').sum().show()

+---------+-----+-----------+--------+
|     dept|state|sum(salary)|sum(age)|
+---------+-----+-----------+--------+
|    Sales|   NY|      17600|      90|
|    Sales|   CA|       8100|      30|
|  Finance|   CA|      18900|      64|
|  Finance|   NY|      16200|      89|
|Marketing|   CA|       8000|      25|
|Marketing|   NY|       9100|      50|
+---------+-----+-----------+--------+



# groupBy with agg
- to apply multiple agg functions

In [22]:
from pyspark.sql.functions import *
df.groupBy(col('dept')).agg(count('salary'),min('salary'),max('salary'),sum('salary')).show()

+---------+-------------+-----------+-----------+-----------+
|     dept|count(salary)|min(salary)|max(salary)|sum(salary)|
+---------+-------------+-----------+-----------+-----------+
|    Sales|            3|       8100|       9000|      25700|
|  Finance|            4|       7900|       9900|      35100|
|Marketing|            2|       8000|       9100|      17100|
+---------+-------------+-----------+-----------+-----------+



# groupBy with Pivot
- convert the state column as headers and apply sum on salary 

In [23]:
df.groupBy('dept').pivot('state').sum('salary').show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [26]:
spark.stop()